# Single QPU VQE experiment (Manual Version)

This notebook goes through the running of a VQE algorithm using interlin-q. For simplicity, we only try out using one QPU. This version goes through the calculation of the expectation values manually, by explicitly calling the backend and passing the density matrices and the list of observables.

For an example of how to use the networking protocols to calculate expectation values individually on the different QPUs and then sending back the results to the controller host, check out the other notebooks, with the name "networking".

## Step 1: Import libraries.

First we import all the necessary libraries. Interlin-q is build using the python framework [QuNetSim](https://arxiv.org/abs/2003.06397) which is a python software framework that can be used to simulate quantum networks up to the network layer. We also need PennyLane's chemistry library for decomposing the Hamiltonian. We would also use PennyLane for the optimiser component

In [1]:
# Basic Libraries
import sys
from pennylane import numpy as np
sys.path.append("../../")

# QuNetSim Components
from qunetsim.components import Network
from qunetsim.objects import Logger
from qunetsim.backends.qutip_backend import QuTipBackend

# Interlin-q Components
from interlinq import (ControllerHost, Constants, Clock,
Circuit, Layer, ComputingHost, Operation)

# Extra needed components
from pennylane import GradientDescentOptimizer
from hamiltonian_decomposition import decompose

Logger.DISABLED = False

## Step 2: Decompose the Hamiltonian.

In [2]:
geometry = 'h2.xyz'
charge = 0
multiplicity = 1
basis_set = 'sto-3g'
name = 'h2'

In [3]:
coefficients, observables, qubit_num = decompose(name, geometry, charge, multiplicity, basis_set)
terms = list(zip(coefficients, observables))

terms

[(-0.04207897647782276, [('Identity', 0)]),
 (0.17771287465139946, [('PauliZ', 0)]),
 (0.1777128746513994, [('PauliZ', 1)]),
 (-0.24274280513140462, [('PauliZ', 2)]),
 (-0.24274280513140462, [('PauliZ', 3)]),
 (0.17059738328801052, [('PauliZ', 0), ('PauliZ', 1)]),
 (0.04475014401535161,
  [('PauliY', 0), ('PauliX', 1), ('PauliX', 2), ('PauliY', 3)]),
 (-0.04475014401535161,
  [('PauliY', 0), ('PauliY', 1), ('PauliX', 2), ('PauliX', 3)]),
 (-0.04475014401535161,
  [('PauliX', 0), ('PauliX', 1), ('PauliY', 2), ('PauliY', 3)]),
 (0.04475014401535161,
  [('PauliX', 0), ('PauliY', 1), ('PauliY', 2), ('PauliX', 3)]),
 (0.12293305056183798, [('PauliZ', 0), ('PauliZ', 2)]),
 (0.1676831945771896, [('PauliZ', 0), ('PauliZ', 3)]),
 (0.1676831945771896, [('PauliZ', 1), ('PauliZ', 2)]),
 (0.12293305056183798, [('PauliZ', 1), ('PauliZ', 3)]),
 (0.17627640804319591, [('PauliZ', 2), ('PauliZ', 3)])]

## Step 3: Prepare Circuit for Given Parameters

The circuit can be prepared using two different ways: either as one circuit, or several circuits run sequentially. The former approach is simpler and generally better for the optimisation function. The latter is better for debugging and for dynamic components of a quantum circuit (i.e. circuits that have a lot of changing operations).

### Main Blocks

In [4]:
def rotational_gate(params):
    phi, theta, omega = params
    cos = np.cos(theta / 2)
    sin = np.sin(theta / 2)
    
    res = np.array([[np.exp(-1j * (phi + omega) / 2) * cos, -np.exp(1j * (phi - omega) / 2) * sin], 
                     [np.exp(-1j * (phi - omega) / 2) * sin, np.exp(1j * (phi + omega) / 2) * cos]])
    
    return res.unwrap()

In [5]:
def initialisation_operations(q_map):
    ops = []
    host_id = list(q_map.keys())[0]
    
    op = Operation(
        name=Constants.PREPARE_QUBITS,
        qids=q_map[host_id],
        computing_host_ids=[host_id])
    ops.append(op)
    
    # Prepare the qubits on the computing host
    op = Operation(
        name=Constants.SINGLE,
        qids=[q_map[host_id][0]],
        gate=Operation.X,
        computing_host_ids=[host_id])
    ops.append(op)
    
    op = Operation(
        name=Constants.SINGLE,
        qids=[q_map[host_id][1]],
        gate=Operation.X,
        computing_host_ids=[host_id])
    ops.append(op)
    
    return Layer(ops)

In [6]:
def ansatz_operations(q_map, parameters):
    layers = []
    host_id = list(q_map.keys())[0]
    
    ops = []
    
    for i in range(len(q_map[host_id])):
        op = Operation(
            name=Constants.SINGLE,
            qids=[q_map[host_id][i]],
            gate=Operation.CUSTOM,
            gate_param=rotational_gate(parameters[i]),
            computing_host_ids=[host_id])
        
        ops.append(op)
        
    layers.append(Layer(ops))
    
    op = Operation(
        name=Constants.TWO_QUBIT,
        qids=[q_map[host_id][2], q_map[host_id][3]],
        gate=Operation.CNOT,
        computing_host_ids=[host_id])
    
    layers.append(Layer([op]))
    
    op = Operation(
        name=Constants.TWO_QUBIT,
        qids=[q_map[host_id][2], q_map[host_id][0]],
        gate=Operation.CNOT,
        computing_host_ids=[host_id])
    
    layers.append(Layer([op]))
    
    op = Operation(
        name=Constants.TWO_QUBIT,
        qids=[q_map[host_id][3], q_map[host_id][1]],
        gate=Operation.CNOT,
        computing_host_ids=[host_id])
    
    layers.append(Layer([op]))
        
    return layers

In [7]:
def measurement_operations(q_map):
    ops = []
    # Measuring only the first qubit
    op = Operation(
        name=Constants.MEASURE,
        qids=['q_0_0'],
        cids=['q_0_0'],
        computing_host_ids=[host_id])
    ops.append(op)
    layers.append(Layer(ops))
    
    # Measuring all qubits
    q_ids = q_map[host_id].copy()
    ops = []
    for q_id in q_ids:
        op = Operation(
            name=Constants.MEASURE,
            qids=[q_id],
            cids=[q_id],
            computing_host_ids=[computing_host_ids[0]])
        ops.append(op)
        
    return Layer(ops)

### Single-step approach

In [8]:
def initialise_and_create_ansatz(q_map, parameters):
    layers = []
    host_id = list(q_map.keys())[0]
    
    # Initialise the qubits on the computing host
    layers.append(initialisation_operations(q_map))
    
    # Apply the ansatz
    layers = layers + ansatz_operations(q_map, parameters)
    
    circuit = Circuit(q_map, layers)
    return circuit

In [9]:
def controller_host_protocol(host, q_map, params):
    """
    Protocol for the controller host
    """
    
    monolithic_circuit = initialise_and_create_ansatz(q_map, params)

    host.generate_and_send_schedules(monolithic_circuit)

In [10]:
def computing_host_protocol(host):
    host.receive_schedule()

### Split-steps approach

In [11]:
def prepare_qubits(q_map): 
    circuit = Circuit(q_map, [initialisation_operations(q_map)])
    return circuit

In [12]:
def apply_ansatz(q_map, parameters):
    circuit = Circuit(q_map, ansatz_operations(q_map, parameters))
    return circuit

In [13]:
def controller_host_protocol_preparation(host, q_map, params):
    """
    Protocol for the controller host
    """
    host.generate_and_send_schedules(prepare_qubits(q_map))

In [14]:
def controller_host_protocol_ansatz(host, q_map, params):
    """
    Protocol for the controller host
    """
    host.generate_and_send_schedules(apply_ansatz(q_map, params))

In [15]:
# Used for the last one
def computing_host_protocol(host):
    host.receive_schedule()

## Step 4: Run the circuit and get the Expectation Value

In [16]:
def init_network():
    network = Network.get_instance()
    network.delay = 0
    network.start()

    clock = Clock()

    qutip = QuTipBackend()

    controller_host = ControllerHost(
        host_id="host_1",
        clock=clock, 
        backend=qutip
    )

    computing_hosts, q_map = controller_host.create_distributed_network(
        num_computing_hosts=1,
        num_qubits_per_host=4)
    controller_host.start()

    network.add_hosts([
        computing_hosts[0],
        controller_host])
    
    return clock, controller_host, computing_hosts, q_map

In [17]:
params = np.random.rand(4, 3)

### Running for the single-step approach

In [18]:
clock, controller_host, computing_hosts, q_map = init_network()

t1 = controller_host.run_protocol(
    controller_host_protocol,
    (q_map, params))
t2 = computing_hosts[0].run_protocol(computing_host_protocol)

t1.join()
t2.join()

INFO:qu_net_sim:Host QPU_0 started processing
INFO:qu_net_sim:Host host_1 started processing
INFO:qu_net_sim:host_1 sends BROADCAST message
INFO:qu_net_sim:sending ACK:1 from QPU_0 to host_1
INFO:qu_net_sim:QPU_0 received {"QPU_0": [{"name": "PREPARE_QUBITS", "qids": ["q_0_0", "q_0_1", "q_0_2", "q_0_3"], "cids": null, "gate": null, "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 0}, {"name": "SINGLE", "qids": ["q_0_0"], "cids": null, "gate": "X", "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 0}, {"name": "SINGLE", "qids": ["q_0_1"], "cids": null, "gate": "X", "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 0}, {"name": "SINGLE", "qids": ["q_0_0"], "cids": null, "gate": "custom_gate", "gate_param": [[[0.7837452862561133, -0.6128197342845608], [-0.1007024864903027, -0.007369447563676435]], [[0.1007024864903027, -0.007369447563676435], [0.783

In [19]:
# This should be 7 after the first invocation. 
# For any further invocations, it would increase by 6: 13,19, 25, etc. So is that expected behaviour?
clock.ticks

6

### Running for the multi-step approach

In [20]:
clock, controller_host, computing_hosts, q_map = init_network()

t1 = controller_host.run_protocol(
    controller_host_protocol_preparation,
    (q_map, params))
t2 = computing_hosts[0].run_protocol(computing_host_protocol)

t1.join()
t2.join()

INFO:qu_net_sim:Host QPU_0 started processing
INFO:qu_net_sim:Host host_1 started processing
INFO:qu_net_sim:host_1 sends BROADCAST message
INFO:qu_net_sim:sending ACK:1 from QPU_0 to host_1
INFO:qu_net_sim:QPU_0 received {"QPU_0": [{"name": "PREPARE_QUBITS", "qids": ["q_0_0", "q_0_1", "q_0_2", "q_0_3"], "cids": null, "gate": null, "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 0}, {"name": "SINGLE", "qids": ["q_0_0"], "cids": null, "gate": "X", "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 0}, {"name": "SINGLE", "qids": ["q_0_1"], "cids": null, "gate": "X", "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 0}]} with sequence number 0
INFO:qu_net_sim:QPU_0 sends CLASSICAL to host_1 with sequence 0
INFO:qu_net_sim:host_1 received ACK from QPU_0 with sequence number 0
INFO:qu_net_sim:QPU_0 awaits classical ACK from host_1 with sequence 0
INFO

In [21]:
clock.ticks

2

In [22]:
t1 = controller_host.run_protocol(
    controller_host_protocol_ansatz,
    (q_map, params))
t2 = computing_hosts[0].run_protocol(computing_host_protocol)

t1.join()
t2.join()

INFO:qu_net_sim:host_1 sends BROADCAST message
INFO:qu_net_sim:sending ACK:2 from QPU_0 to host_1
INFO:qu_net_sim:QPU_0 received {"QPU_0": [{"name": "SINGLE", "qids": ["q_0_0"], "cids": null, "gate": "custom_gate", "gate_param": [[[0.7837452862561133, -0.6128197342845608], [-0.1007024864903027, -0.007369447563676435]], [[0.1007024864903027, -0.007369447563676435], [0.7837452862561133, 0.6128197342845608]]], "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 2}, {"name": "SINGLE", "qids": ["q_0_1"], "cids": null, "gate": "custom_gate", "gate_param": [[[0.8227621142103074, -0.4989914478741642], [-0.25250450222209775, -0.10154562878923407]], [[0.25250450222209775, -0.10154562878923407], [0.8227621142103074, 0.4989914478741642]]], "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 2}, {"name": "SINGLE", "qids": ["q_0_2"], "cids": null, "gate": "custom_gate", "gate_param": [[[0.7907308935274813, -0.4756595635842996], [-0.3849122324063409, 

In [23]:
clock.ticks

7

### Using qutip to get the expectation value

In [24]:
from qutip import identity, sigmax, sigmay, sigmaz
from qutip import expect

def string_to_qutip_pauli(string):
    if string == 'Identity':
        return identity(2)
    if string == 'PauliX':
        return sigmax()
    if string == 'PauliY':
        return sigmay()
    if string == 'PauliZ':
        return sigmaz()

def expectation_value(terms, matrices):
    total = 0
    
    for term in terms:
        coefficient, observables = term
        
        needed_matrices = []
        needed_paulis = []
        
        for obs in observables:
            pauli, index = obs
            
            needed_matrices.append(matrices[index])
            needed_paulis.append(string_to_qutip_pauli(pauli))
        
        expectation = expect(needed_paulis, needed_matrices)
        
        total += coefficient * expectation[0][0]
    
    return total

In [25]:
density_matrices = []

for qubit_id in computing_hosts[0].qubit_ids:
    qubit = computing_hosts[0].get_qubit_by_id(qubit_id)
    density_matrices.append(computing_hosts[0].backend.density_operator(qubit))
    
density_matrices

[Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
 Qobj data =
 [[ 0.15566008+0.j         -0.08344124+0.03932433j]
  [-0.08344124-0.03932433j  0.84433992+0.j        ]],
 Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
 Qobj data =
 [[ 0.28788791+0.j         -0.25842154+0.02113984j]
  [-0.25842154-0.02113984j  0.71211209+0.j        ]],
 Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
 Qobj data =
 [[ 0.85150737+0.j         -0.02352667+0.01572123j]
  [-0.02352667-0.01572123j  0.14849263+0.j        ]],
 Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
 Qobj data =
 [[ 0.74899879+0.j         -0.12322411+0.09454659j]
  [-0.12322411-0.09454659j  0.25100121+0.j        ]]]

In [26]:
expectation_value(terms, density_matrices)

-0.8483835029185777

## Optimise

In [27]:
from qutip import identity, sigmax, sigmay, sigmaz
from qutip import expect

def string_to_qutip_pauli(string):
    if string == 'Identity':
        return identity(2)
    if string == 'PauliX':
        return sigmax()
    if string == 'PauliY':
        return sigmay()
    if string == 'PauliZ':
        return sigmaz()

def expectation_value(terms, matrices):
    total = 0
    
    for term in terms:
        coefficient, observables = term
        
        needed_matrices = []
        needed_paulis = []
        
        for obs in observables:
            pauli, index = obs
            
            needed_matrices.append(matrices[index])
            needed_paulis.append(string_to_qutip_pauli(pauli))
        
        expectation = expect(needed_paulis, needed_matrices)
        
        total += coefficient * expectation[0][0]
    
    return total

In [28]:
def cost_fn(params):
    params = params.reshape(4, 3)
    
    network = Network.get_instance()
    network.delay = 0
    network.start()

    clock = Clock()

    qutip = QuTipBackend()

    controller_host = ControllerHost(
        host_id="host_1",
        clock=clock, 
        backend=qutip
    )

    computing_hosts, q_map = controller_host.create_distributed_network(
        num_computing_hosts=1,
        num_qubits_per_host=4)
    controller_host.start()

    network.add_hosts([
        computing_hosts[0],
        controller_host])
    
    t1 = controller_host.run_protocol(
    controller_host_protocol,
    (q_map, params))
    t2 = computing_hosts[0].run_protocol(computing_host_protocol)

    t1.join()
    t2.join()
    
    density_matrices = []

    for qubit_id in computing_hosts[0].qubit_ids:
        qubit = computing_hosts[0].get_qubit_by_id(qubit_id)
        density_matrices.append(computing_hosts[0].backend.density_operator(qubit))
    
    
    network.remove_host(computing_hosts[0])
    network.remove_host(controller_host)
    
    return expectation_value(terms, density_matrices)

In [53]:
np.random.seed(0)
params = np.random.normal(0, np.pi, (4, 3))

params

tensor([[ 5.54193389,  1.25713095,  3.07479606],
        [ 7.03997361,  5.86710646, -3.07020901],
        [ 2.98479079, -0.47550269, -0.32427159],
        [ 1.28993324,  0.45252622,  4.56873497]], requires_grad=True)

In [30]:
cost_fn(params)

INFO:qu_net_sim:Host QPU_0 started processing
INFO:qu_net_sim:Host host_1 started processing
INFO:qu_net_sim:host_1 sends BROADCAST message
INFO:qu_net_sim:sending ACK:1 from QPU_0 to host_1
INFO:qu_net_sim:QPU_0 received {"QPU_0": [{"name": "PREPARE_QUBITS", "qids": ["q_0_0", "q_0_1", "q_0_2", "q_0_3"], "cids": null, "gate": null, "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 0}, {"name": "SINGLE", "qids": ["q_0_0"], "cids": null, "gate": "X", "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 0}, {"name": "SINGLE", "qids": ["q_0_1"], "cids": null, "gate": "X", "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 0}, {"name": "SINGLE", "qids": ["q_0_0"], "cids": null, "gate": "custom_gate", "gate_param": [[[-0.31798493034765196, 0.7437467353318117], [-0.19454774447448653, -0.5548671488518317]], [[0.19454774447448653, -0.5548671488518317], [-0.317

-0.8130467032961869

In [31]:
Logger.DISABLED = True

### SciPy Optimisers

In [32]:
from scipy.optimize import minimize
minimum_bfgs = minimize(cost_fn, params, method="BFGS", tol=1e-4)

minimum_bfgs

      fun: -1.458543801562212
 hess_inv: array([[ 1.00000000e+00, -2.24256717e-05, -8.30487729e-11,
         1.92964970e-09, -3.79517426e-05,  1.92964975e-09,
        -5.82088210e-09, -1.23750444e-06, -5.82088210e-09,
        -8.25245758e-11,  3.43949096e-05, -8.25245756e-11],
       [-2.24256717e-05,  1.61248569e+00, -2.24256717e-05,
        -1.75823127e-06, -1.21038284e-03, -1.75823195e-06,
         4.40454955e-06,  4.71932471e-02,  4.40454955e-06,
        -2.23978712e-05,  3.12751163e-02, -2.23978712e-05],
       [-8.30487729e-11, -2.24256717e-05,  1.00000000e+00,
         1.92964970e-09, -3.79517426e-05,  1.92964975e-09,
        -5.82088210e-09, -1.23750444e-06, -5.82088210e-09,
        -8.25245758e-11,  3.43949096e-05, -8.25245756e-11],
       [ 1.92964970e-09, -1.75823127e-06,  1.92964970e-09,
         9.99999999e-01,  3.74156326e-06, -7.15521429e-10,
         2.02926378e-09, -1.28881277e-06,  2.02926378e-09,
         1.92955923e-09,  4.00004971e-06,  1.92955923e-09],
       [-3.

In [33]:
minimum_bfgs.x.reshape(4,3)

array([[ 5.54193388e+00,  1.38196422e-07,  3.07479605e+00],
       [ 7.03997358e+00,  6.28318811e+00, -3.07020904e+00],
       [ 2.98479085e+00,  1.44175674e-08, -3.24271526e-01],
       [ 1.28993326e+00,  2.51136198e-06,  4.56873498e+00]])

In [34]:
from scipy.optimize import minimize
minimum_powell = minimize(cost_fn, params, method="Powell", tol=1e-4)

minimum_powell

   direc: array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])
     fun: -1.4585438015626593
 message: 'Optimization terminated successfully.'
    nfev: 354
     nit: 2
  status: 0
 success: True
       x: array([ 8.64960745e+00, -3.37645419e-06,  6.50184703e+00,  1.04498037e+01,
        6.28318531e+00,  4.49597326e-01,  4.05775526e+00, -8.15788377e-11,
        2.26

In [35]:
minimum_powell.x.reshape(4,3)

array([[ 8.64960745e+00, -3.37645419e-06,  6.50184703e+00],
       [ 1.04498037e+01,  6.28318531e+00,  4.49597326e-01],
       [ 4.05775526e+00, -8.15788377e-11,  2.26365738e+00],
       [ 6.37968963e-01,  2.44609655e-10,  6.60317682e+00]])

In [36]:
from scipy.optimize import minimize
minimum_cobyla = minimize(cost_fn, params, method="COBYLA", tol=1e-4)

minimum_cobyla

     fun: -1.458543800271009
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 191
  status: 1
 success: True
       x: array([[ 5.55694514e+00, -7.61630109e-06,  4.28383432e+00],
       [ 6.28318768e+00,  5.80744365e+00, -3.31610951e+00],
       [ 2.96686515e+00, -1.04722187e+00,  1.59533893e-05],
       [ 9.63660043e-01,  4.09808099e-05,  4.04340722e+00]])

In [37]:
minimum_cobyla.x.reshape(4,3)

array([[ 5.55694514e+00, -7.61630109e-06,  4.28383432e+00],
       [ 6.28318768e+00,  5.80744365e+00, -3.31610951e+00],
       [ 2.96686515e+00, -1.04722187e+00,  1.59533893e-05],
       [ 9.63660043e-01,  4.09808099e-05,  4.04340722e+00]])

### Scikit-Quant Optimisers

In [39]:
from skquant.interop.scipy import *

In [55]:
bounds = np.array([-3*np.pi, 3*np.pi])
bounds = np.tile(bounds, (4*3, 1))

flattened_parameters = params.flatten()

In [62]:
budget = 200
minimum_imfil = minimize(cost_fn, flattened_parameters, method=imfil, bounds=bounds, options={'budget' : budget})

minimum_imfil

     fun: -1.4445764338083213
 message: 'completed'
    nfev: 211
  status: 0
 success: True
       x: array([ 7.89812838,  6.26098538,  3.07479606,  7.03997361,  6.17929048,
       -3.07020901,  4.16288804, -0.12577207, -0.32427159,  1.58445755,
        0.01826838,  4.56873497])

In [58]:
budget = 100
minimum_bobyqa = minimize(cost_fn, flattened_parameters, method=pybobyqa, bounds=bounds, options={'budget' : budget})

minimum_bobyqa

     fun: -1.4363382013118617
 message: 'completed'
    nfev: 100
  status: 0
 success: True
       x: array([ 5.54193389, -0.1618809 ,  3.07479606,  7.03997361,  6.30027281,
       -3.07020901,  2.98479079,  0.13890872, -0.32427159,  1.28993324,
       -0.02506119,  4.56873497])

In [61]:
budget = 200
minimum_snobfit = minimize(cost_fn, flattened_parameters, method=snobfit, bounds=bounds, options={'budget' : budget})

minimum_snobfit

     fun: -1.3371320476152808
 message: 'completed'
    nfev: 211
  status: 0
 success: True
       x: array([-5.50011192,  0.33891502, -4.96874294,  4.34972352, -6.29650566,
       -4.42851467,  1.59863084, -5.96739241, -3.51657315,  3.89921914,
       -6.50648971, -6.01809772])

### Gradient Descent Optimisers 

In [38]:
%debug
#max_iterations = 200
#conv_tol = 1e-06

#for n in range(max_iterations):
#    params, prev_energy = opt.step_and_cost(cost_fn, params)
#    energy = cost_fn(params)
#    conv = np.abs(energy - prev_energy)

#    if n % 20 == 0:
#        print('Iteration = {:},  Energy = {:.8f} Ha'.format(n, energy))

#    if conv <= conv_tol:
#        break

ERROR:root:No traceback has been produced, nothing to debug.
